# Prediction Part → First Leg of Hypothesis Tests (**Event Factory**)

We are now ready to start **hypothesis testing**.  
But our hypotheses are not “one-line rules”. They describe **specific market situations** that may happen intraday, and we must translate each hypothesis into something the dataset can understand.

That is why we split the whole hypothesis-testing pipeline into **three separate legs** (three notebooks / layers).  
Each layer has *one job* and does not mix responsibilities.

Not every hypothesis is “event → direction prediction”. Some are:
- **comparison hypotheses** (A vs B),
- **grouping hypotheses** (segment/order),
- **moderator hypotheses** (filters that change other hypotheses’ performance).

So the “Event Factory” work is heavier for some hypotheses, lighter for others.

## The 3-Leg Pipeline (How the whole system is organized)

### **Leg 1 — prediction.ipynb (Event Factory)** --> WE ARE IN HERE !!!
**Goal:** Create new columns that mark **where** each hypothesis applies and **what the rule predicts** (if applicable).

- We do **NOT** prove anything here.
- We do **NOT** compute p-values or draw conclusions here.
- We only build:
  - **event masks** (which rows are “in the hypothesis world”)
  - **baseline rule predictions** (optional for some hypotheses)

**Output pattern (typical):**
- `is_Hx` → 0/1 mask for “this row is a valid event for Hx”
- `pred_Hx_*` → rule-based prediction (if the hypothesis is directional)
- additional tags (if the hypothesis is grouping / moderator)

Some hypotheses are already fully defined by existing features and labels → they may pass through this leg with minimal work.

### **Leg 2 — eval_metrics.ipynb (Metrics Engine)**
**Goal:** Given an event definition and predictions, compute all **evaluation metrics** consistently.

Think of this as a reusable “calculator”:

- Input: **(mask, y_true, y_pred)**  
- Output: metrics such as:
  - **hit-rate**
  - **precision / recall / F1** (for rule predictions)
  - **p-value** (example: test `hit-rate > 0.5`)
  - **effect size** (example: signed returns, distance reduction)
  - reusable slice logic (IB width, gap alignment, etc.)

### **Leg 3 — hypothesis_tests.ipynb (Report + Decision Layer)**
**Goal:** Present results in a clean, viewer-friendly form and state decisions clearly.

This is where we:
- show tables/figures
- explain measurement choices (especially for “reversion” type ideas)
- decide:
  - **Reject null hypothesis** or
  - **Fail to reject null hypothesis**
based on the metrics and p-values from Leg 2.


# What we build in the **Prediction Part** (Leg 1) for each hypothesis

In **Leg 1 (Event Factory)**, we do one simple job:

> For each hypothesis, we **mark the rows where the hypothesis applies** (event detection),  
> and (when meaningful) we also create a **very simple rule-based prediction**.

Important boundaries for Leg 1:

- We **do not** calculate p-values or prove anything here.
- We **do not** make final decisions here.
- We only create **columns** that later notebooks can evaluate.

For each hypothesis, we try to produce:

- **`is_Hx`** → a 0/1 mask that answers:  
  **“Does this 1-minute candle belong to hypothesis Hx’s event world?”**
- **`pred_Hx_*`** → a baseline prediction (only if the hypothesis is directional)
- **tags / regime flags** → if the hypothesis is about groups or conditions


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

### **H1: Reversion in between state**

#### **(a) Build the event mask → `is_H1`**
H1 is only valid when **all** of the following are true:

- **`state_ud_between == 1`**  
  → price is **between** the two AVWAP lines (not above both, not below both) --> AVWAP_up, AVWAP_down
      1. Anchored before 5 minute strong up move --> AVWAP_up
      2. Anchored before 5 minute strong down move --> AVWAP_down
      
- **`delta_ud_pct` is small** (we will set a threshold)  
  → These AVWAPs are **close together**, so the “between” state is really meaningful  
  (not a wide, noisy separation) which may show us possible breakout

- **`slope_up_sign` and `slope_down_sign` are opposite**  
  → one AVWAP is drifting up while the other drifts down  
  (this creates a “tug-of-war” environment)

**Output:**
- **`is_H1`** (0/1)


#### **(b) Baseline prediction (direction toward the middle)**
H1 is about **moving back toward the middle**, not just up/down.  
But for a binary baseline, we can approximate “reversion” like this:

- If **`close < ib_mid`** → predict price will move **up toward the middle** → **1**
- If **`close > ib_mid`** → predict price will move **down toward the middle** → **0**

- This logic is directly connects AVWAPs with our IB logic which considers IB_high and IB_low levels as market borders.
- Our market assumption is IB_mid is the balanced (baseline) level in that trading day, so we are expecting mean-reverse to that point

**Outputs:**
- **`pred_H1_dir15`**
- **`pred_H1_dir30`**

In [4]:
PROJECT_ROOT = Path("..").resolve()

DATA_CACHE = PROJECT_ROOT / "data" / "cache"

CACHE_FILE = DATA_CACHE / "spy_1min_et_clean_with_labeled.csv"

df_hypothesis1 = pd.read_csv(CACHE_FILE, parse_dates=['datetime'])

df_hypothesis1.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_ou,cross_av_ou_last5,cross_av_od,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,1,1,1,1,648.42,648.24,0.000247,-0.000031,1,0
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,1,1,1,1,648.28,647.97,0.000015,-0.000463,1,0
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,1,1,1,1,648.11,648.27,-0.000231,0.000015,0,1
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,1,1,1,1,648.57,648.24,0.000262,-0.000247,1,0
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,1,1,1,1,648.66,648.29,-0.000008,-0.000578,0,0


In [8]:
# --- (a) H1 event mask: is_H1 ---
# For state of "delta_ud_pct is small" threshold 
DELTA_UD_PCT_TH = 0.001  # 0.10%  (initial value, we will tune this)

between = (df_hypothesis1["state_ud_between"] == 1) #directly states 
tight   = (df_hypothesis1["delta_ud_pct"].abs() <= DELTA_UD_PCT_TH)

# slope signs {-1,0,+1}. "opposite" means their product equals to -1 (both AVWAPs won't has zero value, both have some signs). 
# 0(zero) represent NaN values
# we are stating condition where AVWAPs have opposite slope signs
opposite = (df_hypothesis1["slope_up_sign"].fillna(0).astype(int) * df_hypothesis1["slope_down_sign"].fillna(0).astype(int) == -1)

df_hypothesis1["is_H1"] = (between & tight & opposite).astype(int) 
# is H1?? --> market close price is between AVWAP lines & AVWAPs are closer than 0.1 percent & slopes of these AVWAPs are opposite

# --- (b) Baseline direction prediction: "revertion IB_mid level" ---
# In H1 states: 
# close < ib_mid => 1 if price is below the ib_mid, reversion will be though up, 
# close > ib_mid => 0 if price is below the ib_mid, reversion will be though down, 
# equal NaN => we can't mean revert if price already in ib_mid
pred_mid = np.where(df_hypothesis1["close"] < df_hypothesis1["ib_mid"], 1,
           np.where(df_hypothesis1["close"] > df_hypothesis1["ib_mid"], 0, np.nan))

# now we will create expected returns if our hypothesis is true
# later, we will compare these to original "dir15" and "dir30" to see our expected return is real
df_hypothesis1["pred_H1_dir15"] = np.where(df_hypothesis1["is_H1"] == 1, pred_mid, np.nan)
# if our is_H1 = 1, we will assign our prediction as pred_mid --> shows where price will go directly
df_hypothesis1["pred_H1_dir30"] = np.where(df_hypothesis1["is_H1"] == 1, pred_mid, np.nan)
# there won't be any prediction sign (NaN) if our row (1-min candle) is NOT SATISTIFYING H1 CONDITION

#REMAINDER: Our "pred_H1_dir'xx'" columns are giving exact labels of 0(zero) and 1 logic.

df_hypothesis1.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30,is_H1,pred_H1_dir15,pred_H1_dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,1,648.42,648.24,0.000247,-0.000031,1,0,0,NaN,NaN
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,1,648.28,647.97,0.000015,-0.000463,1,0,0,NaN,NaN
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,1,648.11,648.27,-0.000231,0.000015,0,1,0,NaN,NaN
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,1,648.57,648.24,0.000262,-0.000247,1,0,0,NaN,NaN
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,1,648.66,648.29,-0.000008,-0.000578,0,0,0,NaN,NaN


In [ ]:
# We need to see this logic works in our sample dataframe

df_hypothesis1.info()
df_hypothesis1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21450 entries, 0 to 21449
Data columns (total 67 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   datetime             21450 non-null  datetime64[ns]
 1   high                 21450 non-null  float64       
 2   low                  21450 non-null  float64       
 3   close                21450 non-null  float64       
 4   Volume               21450 non-null  int64         
 5   hl_pct               21450 non-null  float64       
 6   hl5                  21230 non-null  float64       
 7   hl15                 20680 non-null  float64       
 8   trend_score_m30      19855 non-null  float64       
 9   ib_high              21450 non-null  float64       
 10  ib_low               21450 non-null  float64       
 11  ib_mid               21450 non-null  float64       
 12  ib_width             21450 non-null  float64       
 13  ib_width_type        21450 non-

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30,is_H1,pred_H1_dir15,pred_H1_dir30
count,21450,21450.000000,21450.000000,21450.000000,2.145000e+04,21450.000000,21230.000000,20680.000000,19855.000000,21450.000000,...,21450.000000,20625.000000,19800.000000,20625.000000,19800.000000,21450.000000,21450.000000,21450.000000,423.000000,423.000000
mean,2025-10-15 12:44:30,668.153902,667.867178,668.010579,3.136366e+04,0.000430,0.000427,0.000423,0.526895,669.798091,...,0.086900,668.004258,667.990245,-0.000022,-0.000053,0.486760,0.473986,0.019720,0.541371,0.541371
min,2025-09-08 09:30:00,647.510000,647.220000,647.310000,1.314000e+03,0.000016,0.000050,0.000069,0.000000,649.060000,...,0.000000,647.310000,647.310000,-0.010864,-0.015575,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-09-25 14:22:15,661.330000,660.912250,661.122500,1.461550e+04,0.000209,0.000224,0.000224,0.243865,663.230000,...,0.000000,661.120000,661.107500,-0.000546,-0.000780,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2025-10-15 12:44:30,666.970000,666.660000,666.820000,2.288800e+04,0.000333,0.000346,0.000346,0.546512,670.230000,...,0.000000,666.770000,666.695000,0.000015,0.000045,0.000000,0.000000,0.000000,1.000000,1.000000
75%,2025-11-04 11:06:45,673.120000,672.870000,673.000000,3.698300e+04,0.000543,0.000547,0.000549,0.813333,677.380000,...,0.000000,672.930000,672.900000,0.000550,0.000780,1.000000,1.000000,0.000000,1.000000,1.000000
max,2025-11-21 15:59:00,689.700000,689.520000,689.590000,1.362579e+06,0.005578,0.003361,0.002587,1.000000,689.700000,...,1.000000,689.590000,689.490000,0.008204,0.009183,1.000000,1.000000,1.000000,1.000000,1.000000
std,NaN,9.467176,9.486371,9.477293,3.576077e+04,0.000333,0.000284,0.000271,0.307900,9.540465,...,0.281695,9.470038,9.462866,0.001271,0.001837,0.499836,0.499334,0.139041,0.498876,0.498876


In [ ]:
# Saving H1 events file distinct in 'data/cache'

from pathlib import Path

# 1) Define project root which is the main branch in our repository
PROJECT_ROOT = Path("..").resolve()

# 2) We need to go to data/cache folder so define that pathway
DATA_CACHE = PROJECT_ROOT / "data" / "cache"
DATA_CACHE.mkdir(parents=True, exist_ok=True)

clean_csv_path = DATA_CACHE / "spy_1min_et_with_H1_events.csv"

df_hypothesis1.to_csv(clean_csv_path, index=False)

print("Saved CSV to:", clean_csv_path)

Saved CSV to: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/cache/spy_1min_et_with_H1_events.csv


## **H2: Continuation when both agree**

### **(a) Build the event mask → `is_H2`**
H2 is valid when price is clearly positioned:

- **`state_ud_above == 1` OR `state_ud_below == 1`**  
  → price is above both lines *or* below both lines

- We are considering just AVWAP_up and AVWAP_down anchors again.
- AVWAP_open will be considered in H3.

Tightening condition:
- slopes confirm the same direction and are not zero  
  → `slope_up_sign == slope_down_sign` and not 0  
  (both benchmarks drifting together)

**Output:**
- **`is_H2`**


### **(b) Rule prediction**
This hypothesis naturally produces a direction:

- If **above** → predict **up continuation** → **1**
- If **below** → predict **down continuation** → **0**

**Outputs:**
- **`pred_H2_dir15`**
- **`pred_H2_dir30`**


In [11]:
PROJECT_ROOT = Path("..").resolve()

DATA_CACHE = PROJECT_ROOT / "data" / "cache"

CACHE_FILE = DATA_CACHE / "spy_1min_et_clean_with_labeled.csv"

df_hypothesis2 = pd.read_csv(CACHE_FILE, parse_dates=['datetime'])

df_hypothesis2.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_ou,cross_av_ou_last5,cross_av_od,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,1,1,1,1,648.42,648.24,0.000247,-0.000031,1,0
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,1,1,1,1,648.28,647.97,0.000015,-0.000463,1,0
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,1,1,1,1,648.11,648.27,-0.000231,0.000015,0,1
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,1,1,1,1,648.57,648.24,0.000262,-0.000247,1,0
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,1,1,1,1,648.66,648.29,-0.000008,-0.000578,0,0


In [12]:
# --- (a) H2 event mask: is_H2 ---
# Price position: above OR below to our two AVWAPs.
pos = (df_hypothesis2["state_ud_above"] == 1) | (df_hypothesis2["state_ud_below"] == 1)
# If we get one of them "above all avwaps", "below all avwaps" we define this 1-min candle state as "pos"

# Tightening:  same slope signs in a single row (1-min candle) and not zero (both of them are +1 or -1)
# Moreover to the our price positioning logic, we need to make slope signs equal
# Because if both AVWAPs not showing clearly a direction, our hypothesis will be meaningless
su = df_hypothesis2["slope_up_sign"].fillna(0).astype(int)
sd = df_hypothesis2["slope_down_sign"].fillna(0).astype(int)
confirm = (su == sd) & (su != 0) # Also we are making slope signs not equal to 0(zero) because 
# I don't want any signal in "no AVWAP creation in IB hours" 1-min candles which have "NaN" slope values

df_hypothesis2["is_H2"] = (pos & confirm).astype(int) #if both conditions are okey, we can clearly define a 1-min candle as a H2 candle.
# 1 -> Yes, 0 -> No

# --- (b) Rule prediction (continuation direction) ---
# pred values: above both => price going up (1), below both => price going down (0). If not H2: NaN.
# It is a basic structure of price positioning and we consider if price is validates given conditions
# the price will REMAIN in that conditions in 15, and 30 minutes
pred = np.where(df_hypothesis2["state_ud_above"] == 1, 1,
       np.where(df_hypothesis2["state_ud_below"] == 1, 0, np.nan)) 

# if our candle is no H2 candle, we won't use our prediction
df_hypothesis2["pred_H2_dir15"] = np.where(df_hypothesis2["is_H2"] == 1, pred, np.nan)
df_hypothesis2["pred_H2_dir30"] = np.where(df_hypothesis2["is_H2"] == 1, pred, np.nan)
# these blocks make sure that just H2 candles will have "pred" values

df_hypothesis2.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30,is_H2,pred_H2_dir15,pred_H2_dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,1,648.42,648.24,0.000247,-0.000031,1,0,0,NaN,NaN
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,1,648.28,647.97,0.000015,-0.000463,1,0,0,NaN,NaN
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,1,648.11,648.27,-0.000231,0.000015,0,1,0,NaN,NaN
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,1,648.57,648.24,0.000262,-0.000247,1,0,0,NaN,NaN
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,1,648.66,648.29,-0.000008,-0.000578,0,0,0,NaN,NaN


In [13]:
# We need to see this logic works in our sample dataframe

df_hypothesis2.info()
df_hypothesis2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21450 entries, 0 to 21449
Data columns (total 67 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   datetime             21450 non-null  datetime64[ns]
 1   high                 21450 non-null  float64       
 2   low                  21450 non-null  float64       
 3   close                21450 non-null  float64       
 4   Volume               21450 non-null  int64         
 5   hl_pct               21450 non-null  float64       
 6   hl5                  21230 non-null  float64       
 7   hl15                 20680 non-null  float64       
 8   trend_score_m30      19855 non-null  float64       
 9   ib_high              21450 non-null  float64       
 10  ib_low               21450 non-null  float64       
 11  ib_mid               21450 non-null  float64       
 12  ib_width             21450 non-null  float64       
 13  ib_width_type        21450 non-

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30,is_H2,pred_H2_dir15,pred_H2_dir30
count,21450,21450.000000,21450.000000,21450.000000,2.145000e+04,21450.000000,21230.000000,20680.000000,19855.000000,21450.000000,...,21450.000000,20625.000000,19800.000000,20625.000000,19800.000000,21450.000000,21450.000000,21450.000000,17753.000000,17753.000000
mean,2025-10-15 12:44:30,668.153902,667.867178,668.010579,3.136366e+04,0.000430,0.000427,0.000423,0.526895,669.798091,...,0.086900,668.004258,667.990245,-0.000022,-0.000053,0.486760,0.473986,0.827646,0.555906,0.555906
min,2025-09-08 09:30:00,647.510000,647.220000,647.310000,1.314000e+03,0.000016,0.000050,0.000069,0.000000,649.060000,...,0.000000,647.310000,647.310000,-0.010864,-0.015575,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-09-25 14:22:15,661.330000,660.912250,661.122500,1.461550e+04,0.000209,0.000224,0.000224,0.243865,663.230000,...,0.000000,661.120000,661.107500,-0.000546,-0.000780,0.000000,0.000000,1.000000,0.000000,0.000000
50%,2025-10-15 12:44:30,666.970000,666.660000,666.820000,2.288800e+04,0.000333,0.000346,0.000346,0.546512,670.230000,...,0.000000,666.770000,666.695000,0.000015,0.000045,0.000000,0.000000,1.000000,1.000000,1.000000
75%,2025-11-04 11:06:45,673.120000,672.870000,673.000000,3.698300e+04,0.000543,0.000547,0.000549,0.813333,677.380000,...,0.000000,672.930000,672.900000,0.000550,0.000780,1.000000,1.000000,1.000000,1.000000,1.000000
max,2025-11-21 15:59:00,689.700000,689.520000,689.590000,1.362579e+06,0.005578,0.003361,0.002587,1.000000,689.700000,...,1.000000,689.590000,689.490000,0.008204,0.009183,1.000000,1.000000,1.000000,1.000000,1.000000
std,NaN,9.467176,9.486371,9.477293,3.576077e+04,0.000333,0.000284,0.000271,0.307900,9.540465,...,0.281695,9.470038,9.462866,0.001271,0.001837,0.499836,0.499334,0.377697,0.496879,0.496879


In [14]:
# Saving H2 events file distinct in 'data/cache'

from pathlib import Path

# 1) Define project root which is the main branch in our repository
PROJECT_ROOT = Path("..").resolve()

# 2) We need to go to data/cache folder so define that pathway
DATA_CACHE = PROJECT_ROOT / "data" / "cache"
DATA_CACHE.mkdir(parents=True, exist_ok=True)

clean_csv_path = DATA_CACHE / "spy_1min_et_with_H2_events.csv"

df_hypothesis2.to_csv(clean_csv_path, index=False)

print("Saved CSV to:", clean_csv_path)

Saved CSV to: /Users/canka/Dev/python/DSA210-Project-Can-Karadogan/data/cache/spy_1min_et_with_H2_events.csv


## **H3: Third anchor improves stability** (comparison hypothesis)

H3 is not a single rule.  
It asks a comparison question:

> “Is the approach that includes a third anchor (open anchor) more stable/better than the two-anchor approach?”

- Biggest trick in here is that we got really low frequency of events which are in H1 (423 candles in 21450 candles, which is in .info() and .describe() part in H1)
- This means, comparing H1 with third anchor AVWAP_open approach will not give us reliable results
- So, we are using just H2 events to make comparison with third anchor approach

### **(a) Define the evaluation universe**
Most of the time we compare inside the same world as H2:

- **`is_H3 = is_H2`**

**Output:**
- **`is_H3`**


### **(b) Create two prediction variants (so we can compare them later)**
We generate:

- **Baseline** (already exists): **`pred_H2_dir15`**
- **Alternative**: **`pred_H3_dir15`**  
  → same direction idea, but it also requires confirmation from the **open anchor** logic

Also:
- **`is_H3_strict`** → a narrower subset where the open anchor strongly agrees  
  (more selective events, fewer rows, potentially cleaner signal)

**Outputs:**
- **`pred_H3_dir15`**
- **`is_H3_strict`**

In [16]:
# now we will use our H2 .csv file because this hypothesis will directly use everything what H2 creates
# its for comparison 

PROJECT_ROOT = Path("..").resolve()

DATA_CACHE = PROJECT_ROOT / "data" / "cache"

CACHE_FILE = DATA_CACHE / "spy_1min_et_with_H2_events.csv"

df_hypothesis3 = pd.read_csv(CACHE_FILE, parse_dates=['datetime'])

df_hypothesis3.head()

,datetime,high,low,close,Volume,hl_pct,hl5,hl15,trend_score_m30,ib_high,...,cross_av_od_last5,close_f15,close_f30,ret15,ret30,dir15,dir30,is_H2,pred_H2_dir15,pred_H2_dir30
0,2025-09-08 09:30:00,648.86,648.24,648.260,141588,0.000956,NaN,NaN,NaN,649.06,...,1,648.42,648.24,0.000247,-0.000031,1,0,0,NaN,NaN
1,2025-09-08 09:31:00,648.45,648.15,648.270,42118,0.000463,NaN,NaN,NaN,649.06,...,1,648.28,647.97,0.000015,-0.000463,1,0,0,NaN,NaN
2,2025-09-08 09:32:00,648.46,648.10,648.260,37143,0.000555,NaN,NaN,NaN,649.06,...,1,648.11,648.27,-0.000231,0.000015,0,1,0,NaN,NaN
3,2025-09-08 09:33:00,648.47,648.23,648.400,42231,0.000370,NaN,NaN,NaN,649.06,...,1,648.57,648.24,0.000262,-0.000247,1,0,0,NaN,NaN
4,2025-09-08 09:34:00,648.68,648.32,648.665,23659,0.000555,0.00058,NaN,NaN,649.06,...,1,648.66,648.29,-0.000008,-0.000578,0,0,0,NaN,NaN


In [ ]:
# (a) Evaluation universe: H3 world just includes H2 events
df_hypothesis3["is_H3"] = df_hypothesis3["is_H2"].astype(int)

# (b) AVWAP_Open anchor confirmation: H2's prediction direction is the same with our 1-min candle's avwap_open value?
# - H2 prediction is up (1)  => close > avwap_open confirmation --> if our market close price is also above avwap_open
# - H2 prediction is down(0) => close < avwap_open confirmation --> if our market close price is also below avwap_open
open_agree_15 = ((df_hypothesis3["pred_H2_dir15"] == 1) & (df_hypothesis3["close"] > df_hypothesis3["avwap_open"])) | \
                ((df_hypothesis3["pred_H2_dir15"] == 0) & (df_hypothesis3["close"] < df_hypothesis3["avwap_open"]))

open_agree_30 = ((df_hypothesis3["pred_H2_dir30"] == 1) & (df_hypothesis3["close"] > df_hypothesis3["avwap_open"])) | \
                ((df_hypothesis3["pred_H2_dir30"] == 0) & (df_hypothesis3["close"] < df_hypothesis3["avwap_open"]))

# Alternative prediction: only if there are H2 events and "open_agree" condition satisfies, H2 prediction is okey for our H3, otherwise -> NaN
df_hypothesis3["pred_H3_dir15"] = np.where((df_hypothesis3["is_H3"] == 1) & open_agree_15, df_hypothesis3["pred_H2_dir15"], np.nan)
df_hypothesis3["pred_H3_dir30"] = np.where((df_hypothesis3["is_H3"] == 1) & open_agree_30, df_hypothesis3["pred_H2_dir30"], np.nan)

In [ ]:
#YARIN BAK BUNAAA!!!!!

# More precise approach: our market close price and AVWAP_open 
# 1) open_agree_15
# 2) slope_open_sign yönle uyumlu (up=>+1, down=>-1)
# 3) open'a uzaklık gürültü değil (min gap)
OPEN_GAP_PCT_TH = 0.0005  # 0.05% (istersen sonra tune edeceksin)
open_gap_pct = (df_hypothesis3["close"] - df_hypothesis3["avwap_open"]).abs() / df_hypothesis3["close"]

slope_agree_15 = ((df_hypothesis3["pred_H2_dir15"] == 1) & (df_hypothesis3["slope_open_sign"] == 1)) | \
                 ((df_hypothesis3["pred_H2_dir15"] == 0) & (df_hypothesis3["slope_open_sign"] == -1))

df_hypothesis3["is_H3_strict"] = ((df_hypothesis3["is_H3"] == 1) & open_agree_15 & slope_agree_15 & (open_gap_pct >= OPEN_GAP_PCT_TH)).astype(int)
